# Miniproyecto PLN

## Problema 7

### Autores:
- Moisés Barrios Torres
- Cecilia Diana Albelda
- Elena Marrero Castellano
- Irina Filimonova Sevcenco

### Carga de librerías

Importamos todas las librerías necesarias para la carga de datos, preprocesado, modelado y evaluación de modelos.

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import html, keras, math, nltk, random, re, spacy, string, torch

from collections import Counter, defaultdict
from gensim.utils import simple_preprocess
from gensim.utils import tokenize
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize
from scipy.special import softmax
from sklearn import metrics
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer, LabelEncoder
from sklearn.svm import SVC
from transformers import (AutoModel, AutoModelForSequenceClassification,
                          AutoTokenizer, BertForSequenceClassification,
                          BertTokenizerFast, BertTokenizer,
                          TFAutoModelForSequenceClassification, Trainer,
                          TrainingArguments)
from transformers.file_utils import (is_tf_available, is_torch_available,
                                     is_torch_tpu_available)

### Carga de datos 
Cargamos los datos a partir del fichero csv que previamente hemos creado. El archivo consta de 2989 artículos extraídos mediante técnicas de webscrapping del periódico digital "EL PAÍS".

Recordamos que los artículos están etiquetados cada uno con su categoría correspondientes, siendo estas: 

INTERNACIONAL, DEPORTES, CIENCIA, SOCIEDAD, ECONOMÍA y POLÍTICA.

In [ ]:
tabla = pd.read_table('articulos.csv', sep=',', index_col=0)
tabla.head()

,CATEGORÍA,ENLACE,TEXTO
0,INTERNACIONAL,https://elpais.com/television/2021-05-20/la-di...,El Festival de Eurovisión es una tradición mus...
1,INTERNACIONAL,https://elpais.com/television/2021-05-20/eurov...,La recta final del Festival de Eurovisión 2021...
2,INTERNACIONAL,https://elpais.com/economia/2021-05-20/crecen-...,Se dice que el mercado de divisas nunca duerme...
3,INTERNACIONAL,https://elpais.com/economia/2021-05-20/la-puja...,El precio de las casas se ha disparado en país...
4,INTERNACIONAL,https://elpais.com/opinion/2021-05-20/drama-y-...,Me ha sorprendido el tono dramático que ha uti...


Nos guardamos los artículos y las etiquetas

In [ ]:
articulos, labels = tabla['TEXTO'], tabla['CATEGORÍA']

In [ ]:
articulos

0       El Festival de Eurovisión es una tradición mus...
1       La recta final del Festival de Eurovisión 2021...
2       Se dice que el mercado de divisas nunca duerme...
3       El precio de las casas se ha disparado en país...
4       Me ha sorprendido el tono dramático que ha uti...
                              ...                        
2984    Una investigación sorprendente ha unido en el ...
2985    Hace más de 25 años, un grupo de arqueólogos y...
2986    Los dinosaurios, aunque sean fósiles o recreac...
2987    En el interior de una piedra que no mide mucho...
2988    En lo que hoy es el norte de Myanmar se han en...
Name: TEXTO, Length: 2989, dtype: object

In [ ]:
labels

0       INTERNACIONAL
1       INTERNACIONAL
2       INTERNACIONAL
3       INTERNACIONAL
4       INTERNACIONAL
            ...      
2984          CIENCIA
2985          CIENCIA
2986          CIENCIA
2987          CIENCIA
2988          CIENCIA
Name: CATEGORÍA, Length: 2989, dtype: object

### Preprocesado

Los algoritmos de ML funcionan sobre un conjunto de características de entrada definido (features), normalmente numérico. Para procesar texto es necesario convertirlo a una matriz numérica (vector space model). Es conveniente limpiar y normalizar el texto en lenguaje natural antes de convertirlo a una matriz numérica.

Nosotros hemos probado los siguientes prerocesados: 

- **PREPROCESADO 1** (*preprocesado*): quitamos los dígitos, los signos de puntuación, las stopwords y las palabras que tengan una longitud de 1. Además nos quedamos con los lemas en minúscula. Finalmente, la salida es una cadena de texto con los tokens filtrados.

- **PREPROCESADO 2** (normaliza_bis): añadimos un espacio después de "." y "?". Quitamos los signos de puntuación y las stopwords con la librería sklearn. Además nos quedamos con los lemas en minúscula. Finalmente, la salida es una cadena de texto con los tokens filtrados.

- **PREPROCESADO 3** (extraer_adj): añadimos un espacio después de "." y "?" y obtenemos el lema de todos los tokens de tipo ADJ. Finalmente, la salida es una cadena de texto con los tokens filtrados.

- **PREPROCESADO 4** (extraer_noun): añadimos un espacio después de "." y "?" y obtenemos el lema de todos los tokens de tipo NOUN. Finalmente, la salida es una cadena de texto con los tokens filtrados.

- **PREPROCESADO 5** (normaliza_bis_mezcla): añadimos un espacio después de "." y "?". Posteriormente, quitamos los dígitos, los signos de puntuación, las stopwords y las palabras que tengan una longitud de 1. Finalmente, la salida es una cadena de texto con los tokens filtrados lematizados y en minúsculas.

- **PREPROCESADO 6** (simple_preprocess): separa el texto en tokens, filtra tokens de longitud 2, elimina acentos y signos de puntuación. Finalmente, la salida es el texto en minúsculas.

- **PREPROCESADO 7** (tokenize): genera de forma iterativa tokens como cadenas Unicode, eliminando las marcas de acento y, opcionalmente, poniendo en minúscula. El texto de entrada puede ser unicode o una cadena de bytes codificada en utf8. Los tokens en la salida son secuencias contiguas máximas de caracteres alfabéticos (sin dígitos).


#### Definimos las funciones de preprocesado que hemos creado nosotros:

In [ ]:
def preprocesado(texto):
  '''
   Quitamos dígitos, signos de puntuación, stopwords y las palabras que tengan una longitud de 1. 
   Además, nos quedamos con los lemas en minúscula y devolvemos una cadena de texto 
   con los tokens filtrados.
  '''
  doc = nlp(texto) # objeto nlp
  tokens = [token.lemma_.lower() for token in doc if not token.is_digit and not 
            token.is_punct and not token.is_stop and len(token.text) > 1]
  return ' '.join(tokens)

In [ ]:
def normaliza_bis(texto):
  '''
   Añadimos un espacio después de "." y "?", quitamos signos de puntuación y stopwords
   con la librería sklearn. Además, nos quedamos con los lemas en minúscula y 
   devolvemos una cadena de texto con los tokens filtrados.
  '''
    texto = re.sub(r"([\?\.])", r"\1 ", texto) 
    doc = nlp(texto)
    tokens = [t for t in doc if not t.is_stop and not t.is_punct and (len(t) > 1)] 
    palabras = []
    for t in tokens:
        if t.ent_iob_=='B' and t.ent_type_=='PER':
            palabras.append('persona')
        elif t.ent_iob_=='I' and t.ent_type_=='PER':
            continue
        else:
            palabras.append(t.lower_) 
    salida = ' '.join(palabras) 
    return salida

In [ ]:
def extraer_adj(texto):
  '''
   Añadimos un espacio después de "." y "?", obtenemos el lema de todos los tokens de tipo ADJ
   y devolvemos una cadena de texto con los tokens filtrados.
  '''
    texto = re.sub(r"([\?\.])", r"\1 ", texto)
    doc = nlp(texto)
    tokens = [t.lemma_ for t in doc if t.pos_=='ADJ'] 
    return ' '.join(tokens)

In [ ]:
def extraer_noun(texto):
  '''
   Añadimos un espacio después de "." y "?" y obtenemos el lema de todos los tokens de tipo NOUN.
   Devolvemos una cadena de texto con los tokens filtrados
  '''
    texto = re.sub(r"([\?\.])", r"\1 ", texto) 
    doc = nlp(texto)
    tokens = [t.lemma_ for t in doc if t.pos_=='NOUN'] 
    return ' '.join(tokens)

In [ ]:
def normaliza_bis_mezcla(texto):
  '''
  Añadimos un espacio después de "." y "?", quitamos dígitos, signos de puntuación, stopwords y
  palabras que tengan una longitud de 1. Devolvemos una cadena de texto con los
  tokens filtrados lematizados y en minúsculas.
  '''
    texto = re.sub(r"([\?\.])", r"\1 ", texto) 
    doc = nlp(texto)
    tokens = [token for token in doc if not token.is_digit and 
              not token.is_punct and not token.is_stop and len(token.text) > 1]
    palabras = []
    for t in tokens:
        if t.ent_iob_=='B' and t.ent_type_=='PER':
            palabras.append('persona')
        elif t.ent_iob_=='I' and t.ent_type_=='PER':
            continue
        else:
            palabras.append(t.lower_) 
    salida = ' '.join(palabras) 
    return salida

#### Cargamos el modelo en español de spaCy

In [ ]:
spacy.cli.download("es_core_news_sm")
nlp = spacy.load('es_core_news_sm')

✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_sm')


#### Aplicamos las distintas funciones de preprocesado:

In [ ]:
corpus_preprocesado = [preprocesado(t) for t in articulos] 

In [ ]:
corpus_normaliza_bis = [normaliza_bis(t) for t in articulos]

In [ ]:
corpus_extraer_adj = [extraer_adj(t) for t in articulos]

In [ ]:
corpus_extraer_noun = [extraer_noun(t) for t in articulos]

In [ ]:
corpus_normaliza_bis_mezcla = [normaliza_bis_mezcla(t) for t in articulos]

In [ ]:
corpus_simple_preprocess = [' '.join(a) for a in [simple_preprocess(t) for t in articulos]]

In [ ]:
corpus_tokenize = [' '.join(a) for a in [list(tokenize(t, deacc=True, lowercase=True)) for t in articulos]]

#### Dividimos el conjunto de datos en entrenamiento y test

Procedemos a dividir el conjunto inicial. Por un lado, cogemos 2/3 del conjunto de datos inicial para entrenar, es decir, el conjunto de entrenamiento (siendo train_corpus, train_labels los nombres de dichas variables). Por otro lado, el tercio restante (conjunto de test) para comprobar el correcto funcionamiento del modelo (siendo test_corpus, test_labels los nombres de dichas variables). 

Con el conjunto de test no se trabaja en el entrenamiento, asegurándonos así que el modelo no conozca los datos "reales".

Vamos a realizar la división de los datos para cada tipo de preprocesado definido anteriormente.

In [ ]:
# Preprocesamiento 1: preprocesado
train_corpus_p, test_corpus_p, train_labels_p, test_labels_p = train_test_split(corpus_preprocesado,
                                             labels,
                                             test_size=1/3,
                                             random_state=666)

In [ ]:
# Preprocesamiento 2: normaliza_bis
train_corpus_nb, test_corpus_nb, train_labels_nb, test_labels_nb = train_test_split(corpus_normaliza_bis,
                                             labels,
                                             test_size=1/3,
                                             random_state=666)

In [ ]:
# Preprocesamiento 3: extraer_adj
train_corpus_ea, test_corpus_ea, train_labels_ea, test_labels_ea = train_test_split(corpus_extraer_adj,
                                             labels,
                                             test_size=1/3,
                                             random_state=666)

In [ ]:
# Preprocesamiento 4: extraer_noun
train_corpus_en, test_corpus_en, train_labels_en, test_labels_en = train_test_split(corpus_extraer_noun,
                                             labels,
                                             test_size=1/3,
                                             random_state=666)

In [ ]:
# Preprocesamiento 5: normaliza_bis_mezcla
train_corpus_nbm, test_corpus_nbm, train_labels_nbm, test_labels_nbm = train_test_split(corpus_normaliza_bis_mezcla,
                                             labels,
                                             test_size=1/3,
                                             random_state=666)

In [ ]:
# Preprocesamiento 6: simple_preprocess
train_corpus_s, test_corpus_s, train_labels_s, test_labels_s = train_test_split(corpus_simple_preprocess,
                                             labels,
                                             test_size=1/3,
                                             random_state=666)

In [ ]:
# Preprocesamiento 7: tokenize
train_corpus_t, test_corpus_t, train_labels_t, test_labels_t = train_test_split(corpus_tokenize,
                                             labels,
                                             test_size=1/3,
                                             random_state=666)

### Vectorizadores principales

- Matrices sparse:
  - BOW (Bag Of Words): Cada texto se representa por una fila de la matriz, donde cada elemento (columna) representa la frecuencia de aparición de un término del vocabulario en el texto
  - TF-IDF (Term Frequency–Inverse Document Frequency): Representación sobre un vector BoW donde se aplica una ponderación para descontar los términos más comunes y promover los raros.
- Matrices densas:
  - LSA (Latent Semantic Analysis) para extraer un conjunto de conceptos latentes relacionando documentos.
  - LDA (Latent Dirichlet Allocation), que obtiene la distribución de probabilidad de Dirichlet de las combinaciones de (articulo, categoría).

#### Definimos todos los vectorizadores

In [ ]:
def modelo(train_corpus, test_corpus):
  # Modelo BOW
  matrix_features_train = []
  matrix_features_test = []
  bow_vectorizer = CountVectorizer(min_df = 0.1)
  ## características bag of words
  bow_train_features = bow_vectorizer.fit_transform(train_corpus)  
  bow_test_features = bow_vectorizer.transform(test_corpus)
  matrix_features_train.append(bow_train_features)
  matrix_features_test.append(bow_test_features)


  # Modelo TF-IDF
  tfidf_vectorizer = TfidfTransformer()
  ## características tfidf (a partir del BoW)
  tfidf_train_features = tfidf_vectorizer.fit_transform(bow_train_features)
  tfidf_test_features = tfidf_vectorizer.transform(bow_test_features)  
  matrix_features_train.append(tfidf_train_features)
  matrix_features_test.append(tfidf_test_features)
  

  # Modelo LSA
  tv = TfidfVectorizer()
  svd = TruncatedSVD(n_components=100)
  lsa = make_pipeline(tv, svd, Normalizer(copy=False))
  lsa_train_matrix = lsa.fit_transform(train_corpus)
  lsa_test_matrix = lsa.transform(test_corpus)
  matrix_features_train.append(lsa_train_matrix)
  matrix_features_test.append(lsa_test_matrix)


  # Modelo LDA
  tf_vectorizer = CountVectorizer(min_df = 0.01)
  tf_train = tf_vectorizer.fit_transform(train_corpus)
  tf_test = tf_vectorizer.transform(test_corpus)
  lda = LatentDirichletAllocation(n_components = 6, random_state = 666)
  lda_train_matrix = lda.fit_transform(tf_train)
  lda_test_matrix = lda.transform(tf_test)
  matrix_features_train.append(lda_train_matrix)
  matrix_features_test.append(lda_test_matrix)
  return matrix_features_train, matrix_features_test

### Modelos

#### Definimos funciones para la creación y evaluación de modelos

In [ ]:
def get_metrics(true_labels, predicted_labels):
    """
    Calculamos distintas métricas sobre el
    rendimiento del modelo. Devuelve un diccionario
    con los parámetros medidos
    """

    return {
        'Accuracy': np.round(
                        metrics.accuracy_score(true_labels, 
                                               predicted_labels),
                        3),
        'Precision': np.round(
                        metrics.precision_score(true_labels, 
                                               predicted_labels,
                                               average='weighted',
                                               zero_division=0),
                        3),
    'Recall': np.round(
                        metrics.recall_score(true_labels, 
                                               predicted_labels,
                                               average='weighted',
                                               zero_division=0),
                        3),
    'F1 Score': np.round(
                        metrics.f1_score(true_labels, 
                                               predicted_labels,
                                               average='weighted',
                                               zero_division=0),
                        3)}              

In [ ]:
def train_predict_evaluate_model(classifier, train_features, train_labels, 
                                 test_features, test_labels):
    """
    Función que entrena un modelo de clasificación sobre
    un conjunto de entrenamiento, lo aplica sobre un conjunto
    de test y devuelve la predicción sobre el conjunto de test
    y las métricas de rendimiento
    """

    # genera modelo    
    classifier.fit(train_features, train_labels)
    # predice usando el modelo sobre test
    predictions = classifier.predict(test_features) 
    # evalúa rendimiento de la predicción   
    metricas = get_metrics(true_labels=test_labels, 
                predicted_labels=predictions)
    
    return predictions, metricas  

In [ ]:
def calcular_metricas(lmf_train, lmf_test, train_labels, test_labels):
  """
    Esta función aplica "train_predict_evaluate_model" a cada uno de los modelos,
    realizando así la predicción y evaliación de todos ellos
  """

  modelLR = LogisticRegression(solver='liblinear')
  modelNB = GaussianNB()
  modelSVM = SGDClassifier(loss='hinge', max_iter=1000)
  modelRBFSVM = SVC(gamma='scale', C=2)

  modelos = [('Logistic Regression', modelLR),
             ('Naive Bayes', modelNB),
             ('Linear SVM', modelSVM),
             ('Gauss kernel SVM', modelRBFSVM)]

  metricas = []
  resultados = []


  # Modelos con características BoW
  bow_train_f = lmf_train[0].toarray()
  bow_test_f = lmf_test[0].toarray()
  for m, clf in modelos:
      prediccion, metrica = train_predict_evaluate_model(classifier=clf,
                                            train_features=bow_train_f,
                                            train_labels=train_labels,
                                            test_features=bow_test_f,
                                            test_labels=test_labels)
      metrica['modelo']=f'{m} BoW'
      resultados.append(prediccion)
      metricas.append(metrica)


  # Modelos con características TF-IDF
  tfidf_train_f = lmf_train[1].toarray()
  tfidf_test_f = lmf_test[1].toarray()
  for m, clf in modelos:
      prediccion, metrica = train_predict_evaluate_model(classifier=clf,
                                            train_features=tfidf_train_f,
                                            train_labels=train_labels,
                                            test_features=tfidf_test_f,
                                            test_labels=test_labels)
      metrica['modelo']=f'{m} TF-IDF'
      resultados.append(prediccion)
      metricas.append(metrica)


  # Modelos con características LSA
  lsa_train_features = lmf_train[2]
  lsa_test_features = lmf_test[2]
  for m, clf in modelos:
      prediccion, metrica = train_predict_evaluate_model(classifier=clf,
                                            train_features=lsa_train_features,
                                            train_labels=train_labels,
                                            test_features=lsa_test_features,
                                            test_labels=test_labels)
      metrica['modelo']=f'{m} LSA'
      resultados.append(prediccion)
      metricas.append(metrica)


  # Modelos con características LDA
  lda_train_features = lmf_train[3]
  lda_test_features = lmf_test[3]
  for m, clf in modelos:
      prediccion, metrica = train_predict_evaluate_model(classifier=clf,
                                            train_features=lda_train_features,
                                            train_labels=train_labels,
                                            test_features=lda_test_features,
                                            test_labels=test_labels)
      metrica['modelo']=f'{m} LDA'
      resultados.append(prediccion)
      metricas.append(metrica)

  return metricas, resultados

#### Aplicamos los modelos para cada preprocesado



In [ ]:
# Preprocesamiento 1: preprocesado
mf_train_p, mf_test_p = modelo(train_corpus_p, test_corpus_p)

In [ ]:
# Preprocesamiento 2: normaliza_bis
mf_train_nb, mf_test_nb = modelo(train_corpus_nb, test_corpus_nb)

In [ ]:
# Preprocesamiento 3: extraer_adj
mf_train_ea, mf_test_ea = modelo(train_corpus_ea, test_corpus_ea)

In [ ]:
# Preprocesamiento 4: extraer_noun
mf_train_en, mf_test_en = modelo(train_corpus_en, test_corpus_en)

In [ ]:
# Preprocesamiento 5: normaliza_bis_mezcla
mf_train_nbm, mf_test_nbm = modelo(train_corpus_nbm, test_corpus_nbm)

In [ ]:
# Preprocesamiento 6: simple_preprocess
mf_train_s, mf_test_s = modelo(train_corpus_s, test_corpus_s)

In [ ]:
# Preprocesamiento 7: tokenize
mf_train_t, mf_test_t = modelo(train_corpus_t, test_corpus_t)

#### Evaluación de los modelos

In [ ]:
# Preprocesamiento 1: preprocesado
metricas_p, resultados_p = calcular_metricas(mf_train_p, 
                                             mf_test_p, 
                                             train_labels_p, 
                                             test_labels_p)
metricas_df_p = pd.DataFrame(metricas_p)

In [ ]:
# Preprocesamiento 2: normaliza_bis
metricas_nb, resultados_nb = calcular_metricas(mf_train_nb, 
                                             mf_test_nb, 
                                             train_labels_nb, 
                                             test_labels_nb)
metricas_df_nb = pd.DataFrame(metricas_nb)

In [ ]:
# Preprocesamiento 3: extraer_adj
metricas_ea, resultados_ea = calcular_metricas(mf_train_ea, 
                                             mf_test_ea, 
                                             train_labels_ea, 
                                             test_labels_ea)
metricas_df_ea = pd.DataFrame(metricas_ea)

In [ ]:
# Preprocesamiento 4: extraer_noun
metricas_en, resultados_en = calcular_metricas(mf_train_en, 
                                             mf_test_en, 
                                             train_labels_en, 
                                             test_labels_en)
metricas_df_en = pd.DataFrame(metricas_en)

In [ ]:
# Preprocesamiento 5: normaliza_bis_mezcla
metricas_nbm, resultados_nbm = calcular_metricas(mf_train_nbm, 
                                             mf_test_nbm, 
                                             train_labels_nbm, 
                                             test_labels_nbm)
metricas_df_nbm = pd.DataFrame(metricas_nbm)

In [ ]:
# Preprocesamiento 6: simple_preprocess
metricas_s, resultados_s = calcular_metricas(mf_train_s, 
                                             mf_test_s, 
                                             train_labels_s, 
                                             test_labels_s)
metricas_df_s = pd.DataFrame(metricas_s)

In [ ]:
# Preprocesamiento 7: tokenize
metricas_t, resultados_t = calcular_metricas(mf_train_t, 
                                             mf_test_t, 
                                             train_labels_t, 
                                             test_labels_t)
metricas_df_t = pd.DataFrame(metricas_t)

Mostramos las métricas de los tres mejores modelos obtenidas con cada uno de los preprocesados:

In [ ]:
print("preprocesado \n")
print(metricas_df_p.sort_values("Accuracy", ascending=False).head(3))
print("\n")
print("normaliza_bis \n")
print(metricas_df_nb.sort_values("Accuracy", ascending=False).head(3))
print("\n")
print("extraer_adj \n")
print(metricas_df_ea.sort_values("Accuracy", ascending=False).head(3))
print("\n")
print("extraer_noun \n")
print(metricas_df_en.sort_values("Accuracy", ascending=False).head(3))
print("\n")
print("normaliza_bis_mezcla \n")
print(metricas_df_nbm.sort_values("Accuracy", ascending=False).head(3))
print("\n")
print("simple_preprocess \n")
print(metricas_df_s.sort_values("Accuracy", ascending=False).head(3))
print("\n")
print("tokenize \n")
print(metricas_df_t.sort_values("Accuracy", ascending=False).head(3))

preprocesado 

    Accuracy  Precision  Recall  F1 Score                   modelo
11     0.849      0.850   0.849     0.847     Gauss kernel SVM LSA
8      0.845      0.846   0.845     0.842  Logistic Regression LSA
10     0.839      0.846   0.839     0.837           Linear SVM LSA


normaliza_bis 

    Accuracy  Precision  Recall  F1 Score                   modelo
10     0.853      0.854   0.853     0.850           Linear SVM LSA
11     0.846      0.846   0.846     0.843     Gauss kernel SVM LSA
8      0.842      0.842   0.842     0.839  Logistic Regression LSA


extraer_adj 

    Accuracy  Precision  Recall  F1 Score                   modelo
11     0.759      0.759   0.759     0.756     Gauss kernel SVM LSA
10     0.742      0.740   0.742     0.734           Linear SVM LSA
8      0.734      0.727   0.734     0.727  Logistic Regression LSA


extraer_noun 

    Accuracy  Precision  Recall  F1 Score                   modelo
11     0.823      0.825   0.823     0.821     Gauss kernel SVM 

Como podemos observar, todas las combinaciones preprocesado-modelo logran valores de "accuracy" muy similares entre sí. Sabemos que esta métrica (el "accuracy") se obtiene dividiendo el total de predicciones correctas de nuestro modelo entre el total de predicciones realizadas, y suele ser la métrica que más se considera en la evaluación de modelos.

Dicha métrica va entre 0.759 (para el modelo Gauss kernel SVM, con el vectorizador LSA y el preprocesado extraer-adj) y  0.853 (para el modelo  Linear SVM, con el vectorizador LSA y el preprocesado normaliza-bis).


#### Conclusiones

Mostramos la matriz de confusión para la mejor combinación de preprocesado y modelo: Linear SVM, con el vectorizador LSA y el preprocesado normaliza-bis

In [ ]:
modelSVM = SGDClassifier(loss='hinge', max_iter=1000)
prediccion, metrica = train_predict_evaluate_model(classifier=modelSVM,
                                            train_features=mf_train_nb[2],
                                            train_labels=train_labels_nb,
                                            test_features=mf_test_nb[2],
                                            test_labels=test_labels_nb)

cm = metrics.confusion_matrix(test_labels_p, prediccion)
pd.DataFrame(cm, index=modelSVM.classes_, columns=modelSVM.classes_)

,CIENCIA,DEPORTES,ECONOMÍA,INTERNACIONAL,POLÍTICA,SOCIEDAD
CIENCIA,158,0,1,5,1,5
DEPORTES,0,175,2,1,0,1
ECONOMÍA,2,1,70,8,1,10
INTERNACIONAL,6,1,16,169,10,7
POLÍTICA,0,0,2,1,192,2
SOCIEDAD,5,3,35,22,6,79


En un caso ideal, toda la matriz de confusión estaría formada por ceros exceptuando la diagonal principal. Esto implicaría que hemos acertado el 100% de predicciones. En dicha matriz, las filas corresponden a las etiquetas reales de los artículos (clasificadas por el propio periódico) y las columnas hacen referencia a las predicciones de las etiquetas realizadas por nuestro modelo.

Como podemos apreciar, en general la matriz evidencia la buena predicción obtenida, pues concentra sus valores más grandes en la diagonal principal. No obstante, detectamos también que el peor funcionamiento del modelo surge con los artículos de 'SOCIEDAD'.

### Otros modelos

Además de los modelos vistos en la asignatura, vamos a probar también con dos nuevos modelos.

#### Modelo MultinomialNaiveBayes 

En este apartado, vamos a implementar la clase "MultinomialNaiveBayes" a partir de la librería NLTK (Natural Languange Toolkit), muy reconocida en el mundo del NLP.

In [ ]:
nltk.download('punkt')

In [ ]:
class MultinomialNaiveBayes:
  """
  Esta clase servirá para definir nuestro modelo MultinomialNaiveBayes con la 
  librería nltk. A partir de las funciones de esta clase podemos tanto ajustar el 
  modelo como predecir los resultados una vez ajustado.
  """
  def __init__(self, classes, tokenizer):
    self.tokenizer = tokenizer
    self.classes = classes
  
  def group_by_class(self, X, y):
    y = [y for y in y]
    data = dict()
    for c in self.classes:
      textos = []
      for labell in range(len(y)):
        if y[labell] == c:
          textos.append(X[labell])
      data[c] = textos
    return data
  
  def fit(self, X, y):
    self.n_class_items = {}
    self.log_class_priors = {}
    self.word_counts = {}
    self.vocab = vocab
    n = len(X)
    grouped_data = self.group_by_class(X, y)
    for c, data in grouped_data.items():
      self.n_class_items[c] = len(data)
      self.log_class_priors[c] = math.log(self.n_class_items[c]/n)
      self.word_counts[c] = defaultdict(lambda: 0)

      for text in data:
        counts = Counter(nltk.word_tokenize(text))
        for word, count in counts.items():
          self.word_counts[c][word] += count
    return self

  def laplace_smoothing(self, word, text_class):
    num = self.word_counts[text_class][word] + 1
    denom = self.n_class_items[text_class] + len(self.vocab)
    return math.log(num/denom)

  def predict(self, X):
    result = []
    for text in X:
      class_scores = {c: self.log_class_priors[c] for c in self.classes}
      words = set(nltk.word_tokenize(text))
      for word in words:
        if word not in self.vocab: continue
        for c in self.classes:
          log_w_given_c = self.laplace_smoothing(word, c)
          class_scores[c] += log_w_given_c
      result.append(max(class_scores, key = class_scores.get))
    return result

Probamos nuestro modelo con el mejor preprocesado obtenido anteriormente (normaliza-bis)

- Vectorizamos el corpus

In [ ]:
vectorizer = CountVectorizer(max_features=3000)
sents_encoded = vectorizer.fit_transform(train_corpus_nb)
counts = sents_encoded.sum(axis = 0).A1
vocab = list(vectorizer.get_feature_names())

- Definimos el modelo

In [ ]:
MNB = MultinomialNaiveBayes(classes = np.unique(labels),
                            tokenizer = Tokenizer()).fit(train_corpus_nb,
                                                         train_labels_nb)

- Realizamos las predicciones

In [ ]:
nb = MNB.predict(test_corpus_nb)

- Mostramos el "accuracy" del modelo

In [ ]:
print("The accuracy of the MNB classifier is",
      accuracy_score(test_labels_nb, nb))

The accuracy of the MNB classifier is 0.6529588766298897


In [ ]:
print("\nThe classification report with metrics -\n",
      classification_report(test_labels_nb, nb))


The classification report with metrics -
                precision    recall  f1-score   support

      CIENCIA       0.95      0.81      0.88       170
     DEPORTES       0.99      0.84      0.91       179
     ECONOMÍA       0.79      0.12      0.21        92
INTERNACIONAL       0.38      0.98      0.55       209
     POLÍTICA       0.95      0.72      0.82       197
     SOCIEDAD       1.00      0.04      0.08       150

     accuracy                           0.65       997
    macro avg       0.84      0.58      0.57       997
 weighted avg       0.83      0.65      0.62       997



#### Conclusiones:

Parece que este modelo no funciona bien en comparación al resto que hemos probado anteriormente.

De nuevo, observamos que el modelo trabaja especialmente mal con el conjunto de artículos de 'SOCIEDAD' y, en menor medida, con los de 'ECONOMÍA'. Por una parte, en ambas categorías observamos un valor muy alto de 'precision', lo cual quiere decir que no predecimos mal los artículos que clasificamos con estas etiquetas. En cambio, cuando nos fijamos en su 'recall' correspondiente, vemos que este tiene un valor muy bajo, lo cual indica que a pesar de tener una precisión alta, hay muchos artículos pertenecientes a estas categorías que hemos clasificado de manera errónea.

#### Modelo bert-base-spanish-cased

Usaremos bert-base-spanish-cased porque es uno de los mejores transformers para español.

Probamos nuestro modelo sin ninguno de los preprocesados definidos anteriormente, ya que el transformer preprocesa los datos internamente.

- Instalamos la librería

In [ ]:
!pip install transformers numpy torch sklearn emoji

- Definimos el modelo y el tokenizer

In [ ]:
model_name = "dccuchile/bert-base-spanish-wwm-cased"
tokenizer = BertTokenizerFast.from_pretrained(model_name, do_lower_case=False)

- Transformamos la columna CATEGORÍA a numérico para poder trabajar con ella

In [ ]:
mapping = {value:i for i, value in enumerate(tabla["CATEGORÍA"].unique())}
tabla["CATEGORÍA"] = [mapping[cat] for cat in tabla["CATEGORÍA"]]
tabla

,CATEGORÍA,ENLACE,TEXTO
0,0,https://elpais.com/television/2021-05-20/la-di...,El Festival de Eurovisión es una tradición mus...
1,0,https://elpais.com/television/2021-05-20/eurov...,La recta final del Festival de Eurovisión 2021...
2,0,https://elpais.com/economia/2021-05-20/crecen-...,Se dice que el mercado de divisas nunca duerme...
3,0,https://elpais.com/economia/2021-05-20/la-puja...,El precio de las casas se ha disparado en país...
4,0,https://elpais.com/opinion/2021-05-20/drama-y-...,Me ha sorprendido el tono dramático que ha uti...
...,...,...,...
2984,5,https://elpais.com/espana/catalunya/2020-04-20...,Una investigación sorprendente ha unido en el ...
2985,5,https://elpais.com/ciencia/2020-04-01/recupera...,"Hace más de 25 años, un grupo de arqueólogos y..."
2986,5,https://elpais.com/economia/2020-03-30/los-din...,"Los dinosaurios, aunque sean fósiles o recreac..."
2987,5,https://elpais.com/ciencia/2020-03-18/un-fosil...,En el interior de una piedra que no mide mucho...


- Dividimos los datos en train y test

In [ ]:
train_text = df["TEXTO"].values
train_labels = df["CATEGORÍA"].values
train_texts, valid_texts, train_labels, valid_labels = train_test_split(list(train_text),
                                                                     list(train_labels),
                                                                     test_size = 0.2)
texts = list(df["TEXTO"])

- Sacamos la media de la longitud de todos los textos para optimizar el modelo. Como mucho medirá el 400 (hemos comprobado que es el máximo que modelo soporta) y si se queda corto hará un padding.

In [ ]:
leng = [len(txt) for txt in texts]
max_length = 400
max_length

400

- Convertimos los textos a secuencia de tokens.

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_length)
valid_encodings = tokenizer(valid_texts, truncation=True, padding=True, max_length=max_length)

- Empaquetamos nuestros datos de texto tokenizados en un objeto Torch

In [ ]:
class DetoxisDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_dataset = DetoxisDataset(train_encodings, train_labels)
valid_dataset = DetoxisDataset(valid_encodings, valid_labels)

- Descargamos y cargamos el modelo BERT y sus pesos previamente entrenados.

In [ ]:
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=6).to("cuda")

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchi

In [ ]:
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  acc = accuracy_score(labels, preds)
  return {'accuracy': acc,}

- Elegimos los parámetros de entrenamiento para nuestro modelo.

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=4,             # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=20,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
    logging_steps=200,               # log & save weights each logging_steps
    evaluation_strategy="steps",     # evaluate each `logging_steps`
    learning_rate = 0.00001
)

In [ ]:
trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=valid_dataset,          # evaluation dataset
    com pute_metrics=compute_metrics,     # the callback that computes metrics of interest
)

- Entrenamos el modelo

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy
200,1.575000,0.991071,0.702341
400,0.645200,0.547331,0.826087
600,0.405900,0.525983,0.841137


TrainOutput(global_step=600, training_loss=0.8753809102376302, metrics={'train_runtime': 926.45, 'train_samples_per_second': 0.648, 'total_flos': 0, 'epoch': 4.0, 'init_mem_cpu_alloc_delta': 0, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': -454930432, 'train_mem_gpu_alloc_delta': 1323734528, 'train_mem_cpu_peaked_delta': 459337728, 'train_mem_gpu_peaked_delta': 8898516992})

- Evaluamos el modelo

In [ ]:
trainer.evaluate()

{'epoch': 4.0,
 'eval_accuracy': 0.8411371237458194,
 'eval_loss': 0.5259833335876465,
 'eval_mem_cpu_alloc_delta': 0,
 'eval_mem_cpu_peaked_delta': 0,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 507055104,
 'eval_runtime': 17.1893,
 'eval_samples_per_second': 34.789}

- Creamos una función que recibirá un texto como entrada y devolverá la predicción del modelo (la categoría) como salida.

In [ ]:
def get_prediction(text,max_length):
    # prepare our text into tokenized sequence
    inputs = tokenizer(text, padding=True, truncation=True, max_length=max_length, return_tensors="pt").to("cuda")
    # perform inference to our model
    outputs = model(**inputs)
    # get output probabilities by doing softmax
    probs = outputs[0].softmax(1)
    # executing argmax function to get the candidate label
    return probs.argmax()

In [ ]:
from sklearn.metrics import f1_score
pred = [int(get_prediction(text, max_length)) for text in valid_texts]
score = f1_score(valid_labels, pred, average='macro')
print(score)

0.8323836685023083


#### Conclusiones:

Con este modelo obtenemos un "accuracy" bastante alto pero, al igual que el modelo creado por nosotros, no supera al "Linear SVM Model".

### Conclusión final 

Recordamos que, tras haber probado distintas combinaciones de preprocesados, vectorizadores y modelos, el mejor resultado ha sido un "accuracy" de 0.853 obtenido con el modelo Linear SVM, el vectorizador LSA y el preprocesado normaliza-bis. Cabe mencionar que los demás modelos obtenidos también han proporcionado un "accuracy" cercano al anterior, alrededor de 0.83 aproximadamente.

Además, destacamos el problema encontrado con la categoría 'SOCIEDAD' en los modelos: "MultinomialNaiveBayes" con NLTK y "Linear SVM, con el vectorizador LSA y el preprocesado normaliza-bis". En ambos hemos encontrado dificultades para clasificar aquellos artículos con esta categoría. Es probable que, aunque no lo hemos comprobado, el resto de modelos también sufrieran en parte una bajada del "accuracy" debido a la dificultad de clasificar los artículos de dicha categoría.